In [2]:
import codecs
import numpy as np
import re

In [3]:
# Opening files cell

text_file = codecs.open( "metro-2033.txt", "r", "utf_8_sig" )
file = codecs.open('text.txt', 'w', "utf_8_sig")
file_no_spaces = codecs.open('text-no-spaces.txt', 'w', "utf_8_sig")

text = text_file.read()
text_file.close()

In [4]:
# Writing files cell
text = text.lower()
text = re.sub('[^а-я]', ' ', text)
text = re.sub('ъ', 'ь', text)
text = re.sub('ё', 'е', text)


text_no_spaces = re.sub('\s+', '', text)
text = re.sub('\s+', ' ', text)

file.write(text)
file_no_spaces.write(text_no_spaces)
file.close()
file_no_spaces.close()

In [5]:
def count_probs(alphab, txt):
    probs = {}
    length = len(txt)
    for symbol in alphab:
        probs[symbol] = txt.count(symbol)/length
    return probs

In [6]:
def count_bigram_probs(alphab, txt):
    alphab_square = [i+j for i in alphab for j in alphab]

    probs = {}
    length = len(txt) - 1
    for bigram in alphab_square:
        probs[bigram] = txt.count(bigram)/length
    return probs


In [7]:
def count_bigram_probs_2(alphab, txt):
    alphab_square = [i+j for i in alphab for j in alphab]

    probs = {}
    length = int(len(txt)/2)
    for i in alphab_square:
        probs[i] = 0

    for i in range(0, len(txt), 2):
        if i < len(txt) - 1:
            probs[txt[i]+txt[i+1]] += 1

    probs = {k: v/length for k, v in probs.items()}
    return probs

In [25]:
def count_entropy(probs, length):
    h = 0
    prob_pos = {i: j for i, j in probs.items() if j != 0}
    for key, value in prob_pos.items():
        h += value * np.log2(value)
    h = (1 / length) * -h

    return h

In [26]:
alphabet = list('абвгдежзийклмнопрстуфхцчшщыьэюя ')
print('With spaces:')
probs_1 = count_probs(alphabet, text)
probs_2_1 = count_bigram_probs(alphabet, text)
probs_2_2 = count_bigram_probs_2(alphabet, text)
n = len(text)
print(f'H1: {count_entropy(probs_1, 1)}, H2_1: {count_entropy(probs_2_1, 2)}, H2_2: {count_entropy(probs_2_2, 2)}')

With spaces:
H1: 4.3719192771302975, H2_1: 3.968171239330339, H2_2: 3.9681105084082304


In [27]:
alphabet_no_spaces =list('абвгдежзийклмнопрстуфхцчшщыьэюя')
print('No spaces:')
probs_1 = count_probs(alphabet_no_spaces, text_no_spaces)
probs_2_1 = count_bigram_probs(alphabet_no_spaces, text_no_spaces)
probs_2_2 = count_bigram_probs_2(alphabet_no_spaces, text_no_spaces)
n = len(text)
print(f'H1: {count_entropy(probs_1, 1)}, H2_1: {count_entropy(probs_2_1, 2)}, H2_2: {count_entropy(probs_2_2, 2)}')

No spaces:
H1: 4.453315120926602, H2_1: 4.136819747421218, H2_2: 4.13641832945434
